O Business Case visa entender de que forma as features existentes, e as possiveis novas features a serem criadas para responder melhor o modelo, quanto ao valor de frete que vem sendo empregado, no histórico de vendas e entregas.

Considerando o resultado de nossas métricas de medição quanto ao valor de frete, como propriedade intelectual de posse da OLIST:
 1. Propomos a utilização destas informações como insumo para negociação com o mercado de serviços de transporte.
 2. Indicar as os melhores locais de origem para postagem de um produto especifico.

In [2]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [3]:
display(dbutils.fs.ls('/mnt/datasets/brazilian-ecommerce/'))

path,name,size
dbfs:/mnt/datasets/brazilian-ecommerce/mkt_datasets_arch.png,mkt_datasets_arch.png,182541
dbfs:/mnt/datasets/brazilian-ecommerce/olist_closed_deals_dataset.csv,olist_closed_deals_dataset.csv,171352
dbfs:/mnt/datasets/brazilian-ecommerce/olist_customers_dataset.csv,olist_customers_dataset.csv,9033957
dbfs:/mnt/datasets/brazilian-ecommerce/olist_ecommerce_datasets_arch.png,olist_ecommerce_datasets_arch.png,209677
dbfs:/mnt/datasets/brazilian-ecommerce/olist_geolocation_dataset.csv,olist_geolocation_dataset.csv,61273883
dbfs:/mnt/datasets/brazilian-ecommerce/olist_marketing_qualified_leads_dataset.csv,olist_marketing_qualified_leads_dataset.csv,703836
dbfs:/mnt/datasets/brazilian-ecommerce/olist_order_items_dataset.csv,olist_order_items_dataset.csv,15438671
dbfs:/mnt/datasets/brazilian-ecommerce/olist_order_payments_dataset.csv,olist_order_payments_dataset.csv,5777138
dbfs:/mnt/datasets/brazilian-ecommerce/olist_order_reviews_dataset.csv,olist_order_reviews_dataset.csv,14409007
dbfs:/mnt/datasets/brazilian-ecommerce/olist_orders_dataset.csv,olist_orders_dataset.csv,17654914


In [4]:
closed_deals = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_closed_deals_dataset.csv', header = True, sep = ',', inferSchema=True)
customers = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_customers_dataset.csv', header = True, sep = ',', inferSchema=True)
geolocation = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_geolocation_dataset.csv', header = True, sep = ',', inferSchema=True)
marketing = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_marketing_qualified_leads_dataset.csv', header = True, sep = ',', inferSchema=True)
order_items = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_order_items_dataset.csv', header = True, sep = ',', inferSchema=True)
order_payments = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_order_payments_dataset.csv', header = True, sep = ',', inferSchema=True)
order_review = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_order_reviews_dataset.csv', header = True, sep = ',', inferSchema=True)
orders = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_orders_dataset.csv', header = True, sep = ',', inferSchema=True)
products = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_products_dataset.csv', header = True, sep = ',', inferSchema=True)
sellers = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_sellers_dataset.csv', header = True, sep = ',', inferSchema=True)
product_category = spark.read.csv('/mnt/datasets/brazilian-ecommerce/product_category_name_translation.csv', header = True, sep = ',', inferSchema=True)

In [5]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in order_items.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(order_items.select(aux))

order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,0,0,0,0,0,0


In [6]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in orders.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(orders.select(aux))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,0,0,160,1783,2965,0


In [7]:
orders = orders.dropDuplicates()

from pyspark.sql.functions import unix_timestamp
orders = orders.withColumn('order_approved_at', unix_timestamp('order_approved_at'))
orders = orders.withColumn('order_delivered_carrier_date', unix_timestamp('order_delivered_carrier_date'))

orders = orders.fillna('NA', subset=['order_approved_at'])
orders = orders.fillna('NA', subset=['order_delivered_carrier_date'])
orders = orders.fillna('NA', subset=['order_delivered_customer_date'])

In [8]:
orders = orders.dropna()
orders.count()

Out[297]: 96461

In [9]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in orders.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(orders.select(aux))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,0,0,0,0,0,0


In [10]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in order_payments.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(order_payments.select(aux))

order_id,payment_sequential,payment_type,payment_installments,payment_value
0,0,0,0,0


In [11]:
order_payments = order_payments.dropDuplicates()

installments = order_payments[['payment_installments']].agg({'payment_installments': 'mean'}).collect()[0][0]
order_payments = order_payments.fillna(installments, subset=['payment_installments'])

valor_pagamentos = order_payments[['payment_value']].agg({'payment_value': 'mean'}).collect()[0][0]
order_payments = order_payments.fillna(valor_pagamentos, subset=['payment_value'])

order_payments = order_payments.dropna()
order_payments.count()

Out[300]: 103886

In [12]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in products.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(products.select(aux))

product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,610,610,610,610,2,2,2,2


In [13]:
lenght = products[['product_name_lenght']].agg({'product_name_lenght': 'mean'}).collect()[0][0]
products = products.fillna(lenght, subset=['product_name_lenght'])

descrip = products[['product_description_lenght']].agg({'product_description_lenght': 'mean'}).collect()[0][0]
products = products.fillna(descrip, subset=['product_description_lenght'])

photo = products[['product_photos_qty']].agg({'product_photos_qty': 'mean'}).collect()[0][0]
products = products.fillna(photo, subset=['product_photos_qty'])

weight = products[['product_weight_g']].agg({'product_weight_g': 'mean'}).collect()[0][0]
products = products.fillna(weight, subset=['product_weight_g'])

lenght_cm = products[['product_length_cm']].agg({'product_length_cm': 'mean'}).collect()[0][0]
products = products.fillna(lenght_cm, subset=['product_length_cm'])

height = products[['product_height_cm']].agg({'product_height_cm': 'mean'}).collect()[0][0]
products = products.fillna(height, subset=['product_height_cm'])

width = products[['product_width_cm']].agg({'product_width_cm': 'mean'}).collect()[0][0]
products = products.fillna(width, subset=['product_width_cm'])

In [14]:
products = products.fillna('NA', subset=['product_category_name'])

In [15]:
products = products.dropDuplicates()
products = products.dropna()
products.count()

Out[304]: 32951

In [16]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in products.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(products.select(aux))

product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,0,0,0,0,0,0,0,0


In [17]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in customers.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(customers.select(aux))

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0,0,0,0


In [18]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in sellers.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(sellers.select(aux))

seller_id,seller_zip_code_prefix,seller_city,seller_state
0,0,0,0


In [19]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in product_category.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(product_category.select(aux))

product_category_name,product_category_name_english
0,0


In [20]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in geolocation.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(geolocation.select(aux))

geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,0,0,0,0


In [21]:
#if geolocation.count() - geolocation.dropDuplicates().count() > 0:

#Customers
df_Geo_Customers = geolocation
df_Geo_Customers = df_Geo_Customers.repartition(2).cache()
df_Geo_Customers.count()
  
df_Geo_Customers = df_Geo_Customers.fillna('NA', subset=['geolocation_zip_code_prefix'])
df_Geo_Customers = df_Geo_Customers.fillna('NA', subset=['geolocation_lat'])
df_Geo_Customers = df_Geo_Customers.fillna('NA', subset=['geolocation_lng'])
  
df_Geo_Customers = df_Geo_Customers.dropna()
df_Geo_Customers = df_Geo_Customers.dropDuplicates()
df_Geo_Customers = df_Geo_Customers.dropDuplicates(['geolocation_zip_code_prefix'])
  
#Sellers  
df_Geo_Sellers   = geolocation
df_Geo_Sellers = df_Geo_Sellers.repartition(2).cache()
df_Geo_Sellers.count()
  
df_Geo_Sellers = df_Geo_Sellers.fillna('NA', subset=['geolocation_zip_code_prefix'])
df_Geo_Sellers = df_Geo_Sellers.fillna('NA', subset=['geolocation_lat'])
df_Geo_Sellers = df_Geo_Sellers.fillna('NA', subset=['geolocation_lng'])
  
df_Geo_Sellers = df_Geo_Sellers.dropna()
df_Geo_Sellers = df_Geo_Sellers.dropDuplicates()
df_Geo_Sellers = df_Geo_Sellers.dropDuplicates(['geolocation_zip_code_prefix'])

#----------------------------------------------------------------------------------------------------------------
#Customers
df_Geo_Customers = df_Geo_Customers.select('geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng')
df_Geo_Customers = df_Geo_Customers.withColumnRenamed("geolocation_zip_code_prefix", "customer_zip_code_prefix")
df_Geo_Customers = df_Geo_Customers.withColumnRenamed("geolocation_lat"            , "geolocation_lat_customer")
df_Geo_Customers = df_Geo_Customers.withColumnRenamed("geolocation_lng"            , "geolocation_lng_customer")

#Sellers
df_Geo_Sellers = df_Geo_Sellers.select('geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng')
df_Geo_Sellers = df_Geo_Sellers.withColumnRenamed("geolocation_zip_code_prefix", "seller_zip_code_prefix")
df_Geo_Sellers = df_Geo_Sellers.withColumnRenamed("geolocation_lat"            , "geolocation_lat_seller")
df_Geo_Sellers = df_Geo_Sellers.withColumnRenamed("geolocation_lng"            , "geolocation_lng_seller")

display(df_Geo_Sellers)


seller_zip_code_prefix,geolocation_lat_seller,geolocation_lng_seller
6654,-23.541778046452446,-46.93002461480863
7240,-23.444072161809686,-46.43744029087079
13289,-23.039901147094497,-46.983502576605666
12940,-23.115630732734584,-46.55374249411723
18800,-23.200761987005745,-49.38013003129772
13840,-22.364391224338657,-46.945218901974464
20735,-22.90158024173358,-43.2796028896648
28024,-21.776857352837187,-41.315651564198966
35694,-20.24790925406447,-43.77254802681044
78400,-14.148263999999944,-56.883454999999955


In [22]:
#Com estes dados eu crio o codigo do estado para melhorar a acuracia de nosso modelo, podendo utilizar assim o estado como feature na regressao linear
import json
import requests
import copy

from pyspark.sql.functions import *
from pyspark.sql.types import *

def jsonToDataFrame(json, schema=None):
  reader = spark.read
  if schema:
    reader.schema(schema)
  return reader.json(sc.parallelize([json]))

response = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/estados')

dfUF = jsonToDataFrame(response.json())

display(dfUF)

id,nome,regiao,sigla
11,Rondônia,"List(1, Norte, N)",RO
12,Acre,"List(1, Norte, N)",AC
13,Amazonas,"List(1, Norte, N)",AM
14,Roraima,"List(1, Norte, N)",RR
15,Pará,"List(1, Norte, N)",PA
16,Amapá,"List(1, Norte, N)",AP
17,Tocantins,"List(1, Norte, N)",TO
21,Maranhão,"List(2, Nordeste, NE)",MA
22,Piauí,"List(2, Nordeste, NE)",PI
23,Ceará,"List(2, Nordeste, NE)",CE


In [23]:
#Customers
tabelaUF = dfUF.select('id', 'nome', 'sigla')
tabelaUF = tabelaUF.withColumnRenamed("id", "codigo_UF_customers")
tabelaUF = tabelaUF.withColumnRenamed("nome", "desc_estado_customers")
tabelaUF = tabelaUF.withColumnRenamed("sigla", "customer_state")
tabelaUF_customers = tabelaUF.select('codigo_UF_customers', 'desc_estado_customers', 'customer_state')
#display(tabelaUF_customers)

'''
#Com estes dados eu crio o codigo do estado para melhorar a acuracia de nosso modelo, podendo utilizar assim o estado como feature na regressao linear
tabelaUF_customers = spark.read.csv('/FileStore/tables/TabelaUF.csv', header = True, sep = ';', inferSchema=True)
tabelaUF_sellers   = spark.read.csv('/FileStore/tables/TabelaUF.csv', header = True, sep = ';', inferSchema=True)

tabelaUF_customers = tabelaUF_customers.withColumnRenamed("Código UF", "codigo_UF_customers")
tabelaUF_customers = tabelaUF_customers.withColumnRenamed("Unidade da Federação", "desc_estado_customers")
tabelaUF_customers = tabelaUF_customers.withColumnRenamed("UF", "customer_state")
'''

#Sellers

tabelaUF = dfUF.select('id', 'nome', 'sigla')
tabelaUF = tabelaUF.withColumnRenamed("id", "codigo_UF_sellers")
tabelaUF = tabelaUF.withColumnRenamed("nome", "desc_estado_sellers")
tabelaUF = tabelaUF.withColumnRenamed("sigla", "seller_state")
tabelaUF_sellers = tabelaUF.select('codigo_UF_sellers', 'desc_estado_sellers', 'seller_state')
#display(tabelaUF_sellers)

'''
#Com estes dados eu crio o codigo do estado para melhorar a acuracia de nosso modelo, podendo utilizar assim o estado como feature na regressao linear
tabelaUF_sellers = tabelaUF_sellers.withColumnRenamed("Código UF", "codigo_UF_sellers")
tabelaUF_sellers = tabelaUF_sellers.withColumnRenamed("Unidade da Federação", "desc_estado_sellers")
tabelaUF_sellers = tabelaUF_sellers.withColumnRenamed("UF", "seller_state")
'''

Out[312]: '\n#Com estes dados eu crio o codigo do estado para melhorar a acuracia de nosso modelo, podendo utilizar assim o estado como feature na regressao linear\ntabelaUF_sellers = tabelaUF_sellers.withColumnRenamed("Código UF", "codigo_UF_sellers")\ntabelaUF_sellers = tabelaUF_sellers.withColumnRenamed("Unidade da Federação", "desc_estado_sellers")\ntabelaUF_sellers = tabelaUF_sellers.withColumnRenamed("UF", "seller_state")\n'

In [24]:
#União das bases "orders" com "order_items" utilizando a coluna "order_id".
data_merge = order_items.join(orders, "order_id",how='left')

data_merge = data_merge.dropDuplicates()

#União das bases "data_merge" com "order_items" utilizando a coluna "order_id".
data_merge = data_merge.join(order_payments, "order_id",how='left')

#União das bases "data_merge" com "products" utilizando a coluna "product_id".
data_merge = data_merge.join(products, "product_id",how='left')

#União das bases "data_merge" com "customers" utilizando a coluna "customer_id".
data_merge = data_merge.join(customers, "customer_id",how='left')

#União das bases "data_merge" com "df_Geo_Customers" utilizando a coluna "customer_zip_code_prefix".
data_merge = data_merge.join(df_Geo_Customers, "customer_zip_code_prefix" ,how='left')

#União das bases "data_merge" com "sellers" utilizando a coluna "seller_id".
data_merge = data_merge.join(sellers, "seller_id" ,how='left')

#União das bases "data_merge" com "df_Geo_Sellers" utilizando a coluna "seller_zip_code_prefix".
data_merge = data_merge.join(df_Geo_Sellers, "seller_zip_code_prefix" ,how='left')

#União das bases "data_merge" com "Tabelas UF" utilizando a coluna "UF".
data_merge = data_merge.join(tabelaUF_customers, "customer_state" ,how='left')

#União das bases "data_merge" com "Tabelas UF" utilizando a coluna "UF".
data_merge = data_merge.join(tabelaUF_sellers, "seller_state" ,how='left')

data_merge = data_merge.repartition(2).cache()
data_merge.count()

display(data_merge)

seller_state,customer_state,seller_zip_code_prefix,seller_id,customer_zip_code_prefix,customer_id,product_id,order_id,order_item_id,shipping_limit_date,price,freight_value,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_city,geolocation_lat_customer,geolocation_lng_customer,seller_city,geolocation_lat_seller,geolocation_lng_seller,codigo_UF_customers,desc_estado_customers,codigo_UF_sellers,desc_estado_sellers
SC,MG,88370,8ae520247981aa06bc94abddf5f46d34,36213,dd1c72ced80394a61dbef422c1c45b6e,cd9895374edea6a67749d331b0b32070,859f516f2fc3f95772e63c5757ab0d5b,1,2017-11-30T14:56:30.000+0000,589.0,20.56,delivered,2017-11-25T14:46:39.000+0000,1511621790,1511792833,2017-12-07T19:27:05.000+0000,2017-12-28T00:00:00.000+0000,1,credit_card,24,609.56,esporte_lazer,44,1449,2,250,20,14,16,7895280f7add30050d1a5c1a81bc63ae,dores de campos,-21.10552376645215,-44.02673157333752,navegantes,-26.894882673092177,-48.650908344427364,31,Minas Gerais,42,Santa Catarina
SC,SP,88359,609e1a9a6c2539919b8205cf7c4e6ff0,6810,b84061fb9715b273145994becbe9b643,8b51c6ccc9c0548c951214e1cd1547ff,be597ae2dfc955f581832e7430e7e293,1,2018-07-17T18:55:12.000+0000,59.9,19.61,delivered,2018-07-14T18:44:35.000+0000,1531594512,1531912980,2018-07-26T16:03:08.000+0000,2018-08-06T00:00:00.000+0000,1,credit_card,7,79.51,cama_mesa_banho,58,678,1,1000,28,14,16,4324df20523951bac9cfeea0c0b99507,embu das artes,-23.644594541202697,-46.80463918526742,brusque,-27.14082339503853,-48.955332903149234,35,São Paulo,42,Santa Catarina
SC,RS,89023,8c16d1f32a54d92897cc437244442e1b,95012,5f5559367c34af05d2b0c45d01b768fc,0fc6914383c2bea767a9562c9d8b43a1,65693cc880b8cf8c5e24798b6e042e0f,1,2018-02-13T14:31:43.000+0000,44.99,15.1,delivered,2018-02-07T14:09:36.000+0000,1518013920,1518107256,2018-02-15T21:08:00.000+0000,2018-03-07T00:00:00.000+0000,1,credit_card,1,60.09,informatica_acessorios,52,644,4,338,32,6,36,4ed7d341dd01b155473c7262debfcacb,caxias do sul,-29.17255172872199,-51.20848645961682,blumenau,-26.95359960936025,-49.08523397233874,43,Rio Grande do Sul,42,Santa Catarina
SC,SP,89041,86ccac0b835037332a596a33b6949ee1,4523,9ddcd7504deb234b14f0d44bb47de04f,f4b952dd7ab601c02bb1bd06395f45f6,4d91816c61cb9557eb207bcbf0929f44,1,2017-06-15T13:10:12.000+0000,78.55,17.8,delivered,2017-06-10T12:53:02.000+0000,1497100212,1497454448,2017-06-19T20:03:40.000+0000,2017-07-05T00:00:00.000+0000,1,credit_card,2,58.71,relogios_presentes,53,301,2,1183,23,5,26,ab0f956f72812ae4636be4e15b58031e,sao paulo,-23.607517736049427,-46.66962493603204,blumenau,-26.91311754053291,-49.119321033188335,35,São Paulo,42,Santa Catarina
SC,GO,88501,eeb6de78f79159600292e314a77cbd18,75925,22e6c1c1596acb84232aa16b3a1f6618,6ae38d49db7fc65ad1779828542aecba,d6fb74265802d3b7f2c16ebb97cbe900,1,2017-05-18T09:10:15.000+0000,219.0,36.85,delivered,2017-05-12T08:59:53.000+0000,1494580215,1494602320,2017-05-29T15:23:51.000+0000,2017-06-09T00:00:00.000+0000,1,credit_card,10,255.85,ferramentas_jardim,42,317,1,2300,23,22,19,7e91ec540af718ab375b1d13271e9543,castelandia,-18.0907957627517,-50.209785147202844,lages,-27.813840831525216,-50.33511625506378,52,Goiás,42,Santa Catarina
SC,RJ,88352,262237171f48e85819065498cac80d7b,21555,a02334ababbac97e012344f330e4fbb0,de7087431dce1f95bf39105df32825cd,26c2b0e7dd3eb988721bb042f783c80e,3,2017-08-08T09:15:10.000+0000,59.9,17.99,delivered,2017-08-02T08:58:27.000+0000,1501846429,1501705984,2017-08-10T18:16:37.000+0000,2017-08-28T00:00:00.000+0000,1,credit_card,10,389.45,esporte_lazer,35,1660,1,200,16,16,16,dfe96ce580a21f177c1d85fe2c78e1ab,rio de janeiro,-22.854362761873098,-43.35715247876394,brusque,-27.08499703719198,-48.90425637829648,33,Rio de Janeiro,42,Santa Catarina


In [25]:
((data_merge.count(), len(data_merge.columns)))
#117604

Out[314]: (117604, 41)

In [26]:
data_merge = data_merge.dropna()
data_merge = data_merge.dropDuplicates()
((data_merge.count(), len(data_merge.columns)))

Out[315]: (114464, 41)

In [27]:
data_merge.dtypes

Out[316]: [('seller_state', 'string'),
 ('customer_state', 'string'),
 ('seller_zip_code_prefix', 'int'),
 ('seller_id', 'string'),
 ('customer_zip_code_prefix', 'int'),
 ('customer_id', 'string'),
 ('product_id', 'string'),
 ('order_id', 'string'),
 ('order_item_id', 'int'),
 ('shipping_limit_date', 'timestamp'),
 ('price', 'double'),
 ('freight_value', 'double'),
 ('order_status', 'string'),
 ('order_purchase_timestamp', 'timestamp'),
 ('order_approved_at', 'bigint'),
 ('order_delivered_carrier_date', 'bigint'),
 ('order_delivered_customer_date', 'timestamp'),
 ('order_estimated_delivery_date', 'timestamp'),
 ('payment_sequential', 'int'),
 ('payment_type', 'string'),
 ('payment_installments', 'int'),
 ('payment_value', 'double'),
 ('product_category_name', 'string'),
 ('product_name_lenght', 'int'),
 ('product_description_lenght', 'int'),
 ('product_photos_qty', 'int'),
 ('product_weight_g', 'int'),
 ('product_length_cm', 'int'),
 ('product_height_cm', 'int'),
 ('product_width_cm', 'int'),
 ('customer_unique_id', 'string'),
 ('customer_city', 'string'),
 ('geolocation_lat_customer', 'double'),
 ('geolocation_lng_customer', 'double'),
 ('seller_city', 'string'),
 ('geolocation_lat_seller', 'double'),
 ('geolocation_lng_seller', 'double'),
 ('codigo_UF_customers', 'bigint'),
 ('desc_estado_customers', 'string'),
 ('codigo_UF_sellers', 'bigint'),
 ('desc_estado_sellers', 'string')]

In [28]:
data_merge.count() 
#113400 valores de frete < 100
#103633 valores de frete < 35 -- Eliminando outliers

Out[317]: 114464

In [29]:
display(data_merge.select('price', 
                          'payment_sequential', 
                          'payment_installments', 
                          'payment_value', 
                          'product_name_lenght', 
                          'product_description_lenght',
                          'product_photos_qty', 
                          'product_weight_g', 
                          'product_length_cm', 
                          'product_height_cm', 
                          'product_width_cm', 
                          'customer_zip_code_prefix', 
                          'freight_value')\
        .summary('count',
                'min',
                '25%',
                'mean',
                'stddev',
                '50%',
                '75%',
                '85%',
                '95%',
                '99%',
                'max')
       )   
       

summary,price,payment_sequential,payment_installments,payment_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,freight_value
count,114464,114464,114464,114464,114464,114464,114464,114464,114464,114464,114464,114464,114464
min,0.85,1,0,0.0,5,4,1,0,7,2,6,1003,0.0
25%,39.9,1,1,60.76,43,348,1,300,18,8,15,11250,13.07
mean,120.10910032848766,1.090980570310316,2.935446952753704,171.9747245422141,48.77820100642997,785.6267298015097,2.201277257478334,2105.8722917249092,30.223502585965893,16.61106548783897,23.049002306402013,35004.60531695555,19.988339128459614
stddev,183.2286786535891,0.6859407678481669,2.7716535412359535,266.49486812014055,9.951172024042284,646.9708508723472,1.7035876735039583,3768.667697325535,16.118971577333813,13.436378263480732,11.705191462660723,29838.258978199643,15.757562137672558
50%,74.9,1,2,108.16,51,608,2,700,25,13,20,24240,16.28
75%,133.2,1,4,188.97,57,978,3,1800,38,20,30,58411,21.17
85%,179.99,1,6,259.85,59,1277,4,3625,45,27,35,79012,26.57
95%,349.9,1,10,512.87,60,2112,6,9800,62,45,45,90630,45.25
99%,890.0,3,10,1205.0,63,3398,8,18500,94,65,61,97574,85.81


In [30]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in data_merge.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(data_merge.select(aux))

seller_state,customer_state,seller_zip_code_prefix,seller_id,customer_zip_code_prefix,customer_id,product_id,order_id,order_item_id,shipping_limit_date,price,freight_value,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_city,geolocation_lat_customer,geolocation_lng_customer,seller_city,geolocation_lat_seller,geolocation_lng_seller,codigo_UF_customers,desc_estado_customers,codigo_UF_sellers,desc_estado_sellers
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:

data_merge = data_merge.fillna('NA', subset=['customer_id'])
data_merge = data_merge.fillna('NA', subset=['order_status'])
data_merge = data_merge.fillna('NA', subset=['order_purchase_timestamp'])
data_merge = data_merge.fillna('NA', subset=['order_approved_at'])
data_merge = data_merge.fillna('NA', subset=['order_delivered_carrier_date'])
data_merge = data_merge.fillna('NA', subset=['order_delivered_customer_date'])
data_merge = data_merge.fillna('NA', subset=['order_estimated_delivery_date'])
data_merge = data_merge.fillna('NA', subset=['payment_sequential'])
data_merge = data_merge.fillna('NA', subset=['payment_type'])
data_merge = data_merge.fillna('NA', subset=['product_category_name'])
data_merge = data_merge.fillna('NA', subset=['customer_unique_id'])
data_merge = data_merge.fillna('NA', subset=['customer_zip_code_prefix'])
data_merge = data_merge.fillna('NA', subset=['customer_city'])
data_merge = data_merge.fillna('NA', subset=['customer_state'])


In [32]:
data_merge = data_merge.dropna()
#data_merge.count()

In [33]:
pandas_data_merge = data_merge.toPandas()

In [34]:
%python
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(20,20)})

#fig, ax = plt.figure()
sns.heatmap(pandas_data_merge.corr(), xticklabels=pandas_data_merge.corr().columns, yticklabels=pandas_data_merge.corr().columns,cmap='RdBu_r',annot=True, cbar=False)
display()

In [35]:
%python
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(20,20)})

#fig, ax = plt.figure()
sns.heatmap(pandas_data_merge.cov(), xticklabels=pandas_data_merge.cov().columns, yticklabels=pandas_data_merge.cov().columns,cmap='RdBu_r',annot=True, cbar=False)
display()

In [36]:
data_merge = data_merge.withColumn("volume", col("product_length_cm") * col("product_height_cm") * col("product_width_cm"))
display(data_merge)

seller_state,customer_state,seller_zip_code_prefix,seller_id,customer_zip_code_prefix,customer_id,product_id,order_id,order_item_id,shipping_limit_date,price,freight_value,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_city,geolocation_lat_customer,geolocation_lng_customer,seller_city,geolocation_lat_seller,geolocation_lng_seller,codigo_UF_customers,desc_estado_customers,codigo_UF_sellers,desc_estado_sellers,volume
SC,RJ,88352,262237171f48e85819065498cac80d7b,21555,a02334ababbac97e012344f330e4fbb0,de7087431dce1f95bf39105df32825cd,26c2b0e7dd3eb988721bb042f783c80e,3,2017-08-08T09:15:10.000+0000,59.9,17.99,delivered,2017-08-02T08:58:27.000+0000,1501846429,1501705984,2017-08-10T18:16:37.000+0000,2017-08-28T00:00:00.000+0000,1,credit_card,10,389.45,esporte_lazer,35,1660,1,200,16,16,16,dfe96ce580a21f177c1d85fe2c78e1ab,rio de janeiro,-22.854362761873098,-43.35715247876394,brusque,-27.08499703719198,-48.90425637829648,33,Rio de Janeiro,42,Santa Catarina,4096
SC,PR,89217,3d0cd21d41671c46f82cd11176bf7277,85819,1cbec9e3f90761ff856fe2487f253cc4,a0f27f57faa493eaa9d1609c45f7d16d,a01b8c1afb8cc652c7d3b3d069951729,1,2018-06-12T11:30:43.000+0000,35.9,19.32,delivered,2018-06-04T11:13:02.000+0000,1528111843,1528270260,2018-06-18T17:56:41.000+0000,2018-07-12T00:00:00.000+0000,1,credit_card,1,55.22,livros_interesse_geral,34,623,5,850,40,5,30,f1bc47b6c804a6fc63a5f584508f0bd5,cascavel,-24.989449011456255,-53.4531990145754,joinville,-26.285093458703344,-48.870102509565115,41,Paraná,42,Santa Catarina,6000
SC,PR,89053,9bcdfa7b615b3abb9461d9e3ad9886ae,86800,4870fd3dd96185117a9165d9be8cf09a,74437e600638a59559c5238c84805b76,90551de7b2d0a7cae5254f30afd7b761,5,2018-03-19T10:30:41.000+0000,14.5,18.23,delivered,2018-03-13T10:15:51.000+0000,1520940039,1521056914,2018-03-20T10:05:23.000+0000,2018-04-05T00:00:00.000+0000,1,credit_card,10,196.38,esporte_lazer,23,182,2,150,20,10,15,a45653bb3070a7504b2805d252c96cba,apucarana,-23.553747560485178,-51.464147868545446,blumenau,-26.876014501529205,-49.08024376158463,41,Paraná,42,Santa Catarina,3000
SC,SC,89251,ce248b21cb2adc36282ede306b7660e5,89208,3bd9cab16ac6be6b2cf08859994a06e3,4f1a7a45b6d43fa7e4958e013923d9e5,36b1f4e20647a8599217e502c9ed45e0,1,2018-04-06T20:09:29.000+0000,54.87,8.65,delivered,2018-04-02T19:44:20.000+0000,1522699769,1522830838,2018-04-07T16:39:22.000+0000,2018-04-12T00:00:00.000+0000,1,credit_card,2,109.55,instrumentos_musicais,41,802,2,617,31,9,31,3baae02c3b608a48d1dfbb678fe30d3a,joinville,-26.34461413266778,-48.834956996393316,jaragua do sul,-26.488821587845074,-49.08281160776692,42,Santa Catarina,42,Santa Catarina,8649
SC,RJ,89204,a3a38f4affed601eb87a97788c949667,21310,a513d9d214219138b0102cefc895d5cf,ba5d5d6bb022a34df6fe5b5e8c8002ad,1b0473c7b9a74f2c6b2ad1098892b3ec,1,2017-12-21T14:13:27.000+0000,99.9,18.27,delivered,2017-12-16T14:02:41.000+0000,1513433607,1513624939,2018-01-05T19:55:00.000+0000,2018-01-16T00:00:00.000+0000,1,credit_card,4,118.17,esporte_lazer,55,1174,3,400,20,6,15,79ca16160b80eeee06751cfe5960b141,rio de janeiro,-22.882037524525607,-43.331533250072106,joinville,-26.29611808881069,-48.855651113653416,33,Rio de Janeiro,42,Santa Catarina,1800
SC,SP,88351,f18112da765e0971ca5d510dec322bdd,4474,5958f5b5fd615a8898caed646014c55b,455819d0d2ccab9c6041aa8de3ae6b16,162e915bc5d7c6b2de7eca743a65b033,1,2018-01-07T22:26:56.000+0000,158.9,58.9,delivered,2018-01-01T22:11:36.000+0000,1514845646,1514989161,2018-01-12T22:10:50.000+0000,2018-01-18T00:00:00.000+0000,1,credit_card,3,217.8,NA,48,771,2,2400,42,15,27,c3b626053c7b4b7f18ce104ad12f09af,sao paulo,-23.696083120376358,-46.64287355066725,brusque,-27.092689155981297,-48.91252079822836,35,São Paulo,42,Sant

In [37]:
df = data_merge.select('volume',               
               'order_approved_at',
               'order_delivered_carrier_date',
               'price', 
               'payment_value',                
               'product_weight_g', 
               'codigo_UF_customers',
               'codigo_UF_sellers',
               'customer_zip_code_prefix',
               'geolocation_lat_customer',
               'geolocation_lng_customer',
               'seller_zip_code_prefix',
               'geolocation_lat_seller',
               'geolocation_lng_seller',
               'freight_value'
              )  

df = df.repartition(2).cache()
df.count()

display(df)

volume,order_approved_at,order_delivered_carrier_date,price,payment_value,product_weight_g,codigo_UF_customers,codigo_UF_sellers,customer_zip_code_prefix,geolocation_lat_customer,geolocation_lng_customer,seller_zip_code_prefix,geolocation_lat_seller,geolocation_lng_seller,freight_value
15000,1510249673,1510418663,85.9,44.23,900,35,35,8032,-23.51684802637195,-46.42529733166944,3461,-23.570172947102417,-46.5265931248053,9.42
7514,1516241499,1516426765,13.5,21.28,350,35,35,9280,-23.64446351041601,-46.514282217323526,3211,-23.601705181516518,-46.55289895989381,7.78
5236,1516215063,1516307066,149.9,159.69,383,35,35,4813,-23.72267925229382,-46.68497013613022,18025,-23.511512808091418,-47.443792002312634,9.79
48070,1535080517,1535381820,122.73,142.5,3500,35,35,1525,-23.56691589592586,-46.637438228691835,11704,-24.022198215650086,-46.47831460851754,19.77
2304,1509053444,1509396930,24.5,36.35,150,35,35,7952,-23.273724228830996,-46.760323434937604,17900,-21.48421781278559,-51.5356721057283,11.85
1800,1517835342,1518139104,29.9,88.0,500,35,41,13070,-22.896424990564935,-47.08106484026778,82940,-25.45707628282069,-49.220690762421356,14.1
36162,1525252499,1525678260,279.9,307.94,3650,31,35,36050,-21.739620656673797,-43.35276443935025,14940,-21.76590242683773,-48.824481207427674,28.04
352,1525769778,1525782360,29.0,132.68,200,33,35,21940,-22.812410000836724,-43.19869858312654,5849,-23.65034498473656,-46.75515749486411,3.8
4256,1534364425,1534514520,355.9,376.49,450,41,35,86010,-23.32279799999999,-51.15510090954452,13170,-22.823844024822737,-47.26994520496454,20.59
36000,1498271481,1498655863,69.9,85.18,5350,35,35,1445,-23.57272211157595,-46.68832731862769,3470,-23.56801202511814,-46.52272253248186,15.28


In [38]:
df = df.dropDuplicates()

In [39]:
df.dtypes

Out[328]: [('volume', 'int'),
 ('order_approved_at', 'bigint'),
 ('order_delivered_carrier_date', 'bigint'),
 ('price', 'double'),
 ('payment_value', 'double'),
 ('product_weight_g', 'int'),
 ('codigo_UF_customers', 'bigint'),
 ('codigo_UF_sellers', 'bigint'),
 ('customer_zip_code_prefix', 'int'),
 ('geolocation_lat_customer', 'double'),
 ('geolocation_lng_customer', 'double'),
 ('seller_zip_code_prefix', 'int'),
 ('geolocation_lat_seller', 'double'),
 ('geolocation_lng_seller', 'double'),
 ('freight_value', 'double')]

In [40]:
#Verificamos aqui que há muitos outiliers de valor maiores que 40, portanto para este caso especifico nos focaremos nos fretes com valores menores que 
display(data_merge.select('freight_value'))

freight_value
17.99
19.32
18.23
8.65
18.27
58.9
15.19
24.69
16.85
16.11


In [41]:
display(data_merge.select('freight_value'))

freight_value
17.99
19.32
18.23
8.65
18.27
58.9
15.19
24.69
16.85
16.11


In [42]:
res_estat = data_merge.select('freight_value').where(data_merge['freight_value'] >= 1)\
        .summary('count',
                'min',
                '25%',
                'mean',
                'stddev',
                '50%',
                '75%',
                '85%',
                '90%',
                '95%',
                '99%',
                'max')

#No intuito de eliminar os outiliers que neste modelo em especifico, são um dos maiores detratores dos melhores resultado de previsao do modelo
#foram selecionados 90% de toda a base de dados, que são os valores de frete entre R$ 1,00 e 34,13    

nMin = res_estat.select(res_estat['freight_value']).where(res_estat['summary'] == 'min' ).collect()[0]['freight_value']
n90 = res_estat.select(res_estat['freight_value']).where(res_estat['summary'] == '90%' ).collect()[0]['freight_value']
#nMaxM = res_estat.select(res_estat['freight_value']).where(res_estat['summary'] == 'max' ).collect()[0]['freight_value']

nMax = n90
#nMax = nMaxM

print('Valor minimo de frete definido: ')
print(nMin)
print('Valor máximo de frete definido: ')
print(nMax)
          
display(res_estat)      

summary,freight_value
count,113930
min,1.0
25%,13.11
mean,20.08167989116125
stddev,15.735210411713274
50%,16.32
75%,21.19
85%,26.62
90%,34.14
95%,45.28


In [43]:
data_merge_total = df

data_merge_total = data_merge_total.repartition(2).cache()
data_merge_total.count()

data_merge_Slice = df.where((df['freight_value'] > nMin) & (df['freight_value'] < nMax))

data_merge_Slice = data_merge_Slice.repartition(2).cache()
data_merge_Slice.count()

#Antes verifico a quantidade registros que serão descartados
print(data_merge_total.count() - data_merge_Slice.count())

df = data_merge_Slice


#data_merge = data_merge.where((data_merge['freight_value'] > nMin) & (data_merge['freight_value'] < 100))

10883

In [44]:
#Verificamos aqui que há muitos outiliers de valor maiores que 40, portanto para este caso especifico nos focaremos nos fretes com valores menores que 
display(data_merge_Slice.select('freight_value'))

freight_value
20.3
15.1
19.32
9.66
13.07
9.89
15.23
17.26
13.68
9.0


In [45]:
pandas_df = data_merge_Slice.toPandas()

In [46]:
%python
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(20,20)})

#fig, ax = plt.figure()
sns.heatmap(pandas_df.corr(), xticklabels=pandas_df.corr().columns, yticklabels=pandas_df.corr().columns,cmap='RdBu_r',annot=True, cbar=False)
display()

In [47]:
%python
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(20,20)})

#fig, ax = plt.figure()
sns.heatmap(pandas_df.cov(), xticklabels=pandas_df.cov().columns, yticklabels=pandas_df.cov().columns,cmap='RdBu_r',annot=True, cbar=False)
display()

In [48]:
#df               = df.drop("product_id_Indx")
#data_merge_total = data_merge_total.drop("product_id_Indx")

In [49]:
from pyspark.ml.feature import VectorAssembler

In [50]:
assembler    = VectorAssembler(inputCols = df.columns[:-1], outputCol  = 'features')
assembler_Tot = VectorAssembler(inputCols = data_merge_total.columns[:-1], outputCol  = 'features')

In [51]:
df_assembler = assembler.transform(df)
df_assembler_Tot = assembler_Tot.transform(data_merge_total)
display(df_assembler)

volume,order_approved_at,order_delivered_carrier_date,price,payment_value,product_weight_g,codigo_UF_customers,codigo_UF_sellers,customer_zip_code_prefix,geolocation_lat_customer,geolocation_lng_customer,seller_zip_code_prefix,geolocation_lat_seller,geolocation_lng_seller,freight_value,features
1428,1524761579,1524838620,189.49,209.79,550,33,35,22410,-22.983754568959622,-43.21316146280802,12924,-22.862060999999997,-46.526436,20.3,"List(1, 14, List(), List(1428.0, 1.524761579E9, 1.52483862E9, 189.49, 209.79, 550.0, 33.0, 35.0, 22410.0, -22.983754568959622, -43.21316146280802, 12924.0, -22.862060999999997, -46.526436))"
1260,1512124254,1512160609,46.49,61.59,300,41,35,86010,-23.32279799999999,-51.15510090954452,4850,-23.753781000000004,-46.67553201165677,15.1,"List(1, 14, List(), List(1260.0, 1.512124254E9, 1.512160609E9, 46.49, 61.59, 300.0, 41.0, 35.0, 86010.0, -23.32279799999999, -51.15510090954452, 4850.0, -23.753781000000004, -46.67553201165677))"
15180,1520697330,1520875121,49.4,68.72,650,33,31,20950,-22.905163222658093,-43.24412499201628,37564,-22.27164789897835,-46.16347079455195,19.32,"List(1, 14, List(), List(15180.0, 1.52069733E9, 1.520875121E9, 49.4, 68.72, 650.0, 33.0, 31.0, 20950.0, -22.905163222658093, -43.24412499201628, 37564.0, -22.27164789897835, -46.16347079455195))"
1666,1533681918,1534239540,129.9,139.56,393,35,35,12310,-23.29390606189049,-45.940567825004344,13324,-23.206566074093192,-47.31913346585179,9.66,"List(1, 14, List(), List(1666.0, 1.533681918E9, 1.53423954E9, 129.9, 139.56, 393.0, 35.0, 35.0, 12310.0, -23.29390606189049, -45.940567825004344, 13324.0, -23.206566074093192, -47.31913346585179))"
4125,1526668528,1526722020,89.99,103.06,850,35,35,7055,-23.469920413266777,-46.56223445337295,14802,-21.765275540827815,-48.171092937698134,13.07,"List(1, 14, List(), List(4125.0, 1.526668528E9, 1.52672202E9, 89.99, 103.06, 850.0, 35.0, 35.0, 7055.0, -23.469920413266777, -46.56223445337295, 14802.0, -21.765275540827815, -48.171092937698134))"
8000,1527077701,1527511080,219.04,457.86,900,35,35,1321,-23.566604741877807,-46.643028641150536,9400,-23.710688825577073,-46.4141448576244,9.89,"List(1, 14, List(), List(8000.0, 1.527077701E9, 1.52751108E9, 219.04, 457.86, 900.0, 35.0, 35.0, 1321.0, -23.566604741877807, -46.643028641150536, 9400.0, -23.710688825577073, -46.4141448576244))"
896,1526978044,1526996400,19.99,35.22,100,53,35,70773,-15.738367500000004,-47.89473975,4102,-23.57890176908647,-46.640154530122864,15.23,"List(1, 14, List(), List(896.0, 1.526978044E9, 1.5269964E9, 19.99, 35.22, 100.0, 53.0, 35.0, 70773.0, -15.738367500000004, -47.89473975, 4102.0, -23.57890176908647, -46.640154530122864))"
19008,1513050630,1513100601,144.99,162.25,1050,33,35,23065,-22.90593355921822,-43.42129173133025,3426,-23.550569408680975,-46.54088389992436,17.26,"List(1, 14, List(), List(19008.0, 1.51305063E9, 1.513100601E9, 144.99, 162.25, 1050.0, 33.0, 35.0, 23065.0, -22.90593355921822, -43.42129173133025, 3426.0, -23.550569408680975, -46.54088389992436))"
28175,1533094298,1533214380,196.8,420.96,1650,35,35,4531,-23.581221213976647,-46.67626966556059,3204,-23.59503718679664,-46.56257710283338,13.68,"List(1, 14, List(), List(28175.0, 1.533094298E9, 1.53321438E9, 196.8, 420.96, 1650.0, 35.0, 35.0, 4531.0, -23.581221213976647, -46.67626966556059, 3204.0, -23.59503718679664, -46.56257710283338))"
4096,1532103922,1532348700,35.0,88.0,550,35,35,2033,-23.502219297795676,-46.61707725229382,5528,-23.594442924621582,-46.75706480621012,9.0,"List(1, 14, List(), List(4096.0, 1.532103922E9, 1.5323487E9, 35.0, 88.0, 550.0, 35.0, 35.0, 2033.0, -23.502219297795676, -46.61707725229382, 5528.0, -23.594442924621582, -46.75706480621012))"


In [52]:
(df_assembler.count(), len(df_assembler.columns))
(df_assembler_Tot.count(), len(df_assembler_Tot.columns))

Out[341]: (103302, 16)

In [53]:
treino, teste = df_assembler.randomSplit([0.7, 0.3], seed=36)
treino_Tot, teste_Tot = df_assembler_Tot.randomSplit([0.7, 0.3], seed=36)

treino = treino.repartition(2).cache()
treino.count()

teste = teste.repartition(2).cache()
teste.count()

treino_Tot = treino_Tot.repartition(2).cache()
treino_Tot.count()

teste_Tot = teste_Tot.repartition(2).cache()
teste_Tot.count()

Out[342]: 30979

In [54]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol= 'freight_value')

modelo = lr.fit(treino)
modelo_Tot = lr.fit(treino_Tot)

In [55]:
print(f'Coeficientes: {modelo.coefficients}')
print(f'pValues: {modelo.summary.pValues}')
print(f'Intercepto: {modelo.intercept}')

Coeficientes: [4.923260927532487e-05,4.043957718924676e-08,-8.462134278109482e-09,0.007991073939693252,0.00027622430954791875,0.0005440113740920747,-0.13049952548990787,-0.09115457054287224,8.576114646446199e-05,0.21847260891927653,0.16467717982504138,4.706391825234638e-05,0.1699212114057592,-0.14944102547448845]
pValues: [0.0, 0.4412616625976309, 0.87221283820841, 0.0, 0.08525704876124296, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Intercepto: -21.404698103072377

In [56]:
print(f'MAE: {modelo.summary.meanAbsoluteError}') #Erro absoluto do modelo
print(f'RMSE: {modelo.summary.rootMeanSquaredError}') #R2 ao quadrado error em valor
print(f'r2 Ajustado: {modelo.summary.r2adj *100}') #As variaveis que nos temos no modelo explicam 53.33% do modelo

'''
Base Line Metrics

MAE: 5.964145570151102
MSE: 119.27973085177
RMSE: 10.921526031272828
r2 Ajustado: 53.32880323011593'
'''

'''
Após a inclusão do codigo do CEP do vendedor.
Antes do corte de dados, dos outiliers do valor do frete

MAE: 5.742249298528482
MSE: 108.08284389752195
RMSE: 10.396289910228646
r2 Ajustado: 56.504091413487714
'''

maeLR_train  = modelo.summary.meanAbsoluteError
rmseLR_train = modelo.summary.rootMeanSquaredError
r2LR_train   = modelo.summary.r2adj

MAE: 2.8105187608948805
RMSE: 3.8990689068315922
r2 Ajustado: 50.41216995912639

In [57]:
resultado_teste = modelo.evaluate(teste)
resultado_teste_Tot = modelo_Tot.evaluate(teste_Tot)

In [58]:
print(f'MAE: {resultado_teste.meanAbsoluteError}')
print(f'RMSE: {resultado_teste.rootMeanSquaredError}')
print(f'r2 Ajustado: {resultado_teste.r2adj}')

maeLR_teste  = resultado_teste.meanAbsoluteError
rmseLR_teste = resultado_teste.rootMeanSquaredError
r2LR_teste   = resultado_teste.r2adj

maeLR_teste_Tot  = resultado_teste_Tot.meanAbsoluteError
rmseLR_teste_Tot = resultado_teste_Tot.rootMeanSquaredError
r2LR_teste_Tot   = resultado_teste_Tot.r2adj

MAE: 2.8392927715351233
RMSE: 3.9651163473027546
r2 Ajustado: 0.49241315887309733

In [60]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

featureIndexer = VectorIndexer(inputCol="features", 
                              outputCol="featureIndexer",
                              maxCategories=100).fit(df_assembler)

featureIndexer_Tot = VectorIndexer(inputCol="features", 
                              outputCol="featureIndexer",
                              maxCategories=100).fit(df_assembler_Tot)

dt = DecisionTreeRegressor(featuresCol="featureIndexer", labelCol="freight_value", maxBins=128)
#dt = DecisionTreeRegressor(featuresCol="featureIndexer", labelCol="freight_value", maxBins=50000)

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])
pipeline_Tot = Pipeline(stages=[featureIndexer_Tot, dt])

modelarvore = pipeline.fit(treino)
modelarvore_Tot = pipeline_Tot.fit(treino_Tot)

predictions = modelarvore.transform(teste)
predictions_Tot = modelarvore_Tot.transform(teste_Tot)

In [61]:
display(predictions.select("prediction", "freight_value", "features"))

prediction,freight_value,features
9.586066563042879,11.73,"List(1, 14, List(), List(20000.0, 1.518045025E9, 1.518207135E9, 65.9, 77.63, 250.0, 35.0, 35.0, 1151.0, -23.53250199140166, -46.655183398840066, 5767.0, -23.633090317589396, -46.759997268731205))"
17.165794663573088,21.47,"List(1, 14, List(), List(14350.0, 1.493826149E9, 1.493828237E9, 119.99, 141.46, 1350.0, 42.0, 31.0, 89287.0, -26.237652633823842, -49.38599561257329, 37564.0, -22.27164789897835, -46.16347079455195))"
17.165794663573088,15.56,"List(1, 14, List(), List(7140.0, 1.493831729E9, 1.494243743E9, 39.0, 54.56, 1000.0, 41.0, 35.0, 80050.0, -25.43482090674591, -49.25340082846679, 12327.0, -23.30593717675828, -45.97121752309212))"
15.386572847682123,15.1,"List(1, 14, List(), List(576.0, 1.499445816E9, 1.499519264E9, 35.0, 50.1, 150.0, 35.0, 31.0, 13566.0, -21.999410549541242, -47.897242066478114, 38440.0, -18.650389455353206, -48.19675571936371))"
12.01333749220212,11.85,"List(1, 14, List(), List(23625.0, 1.498756808E9, 1.498837575E9, 48.9, 60.75, 900.0, 35.0, 35.0, 13504.0, -22.392499159039716, -47.573597665156555, 14940.0, -21.76590242683773, -48.824481207427674))"
17.165794663573088,16.76,"List(1, 14, List(), List(15360.0, 1.519378176E9, 1.519420964E9, 72.9, 89.66, 1500.0, 33.0, 35.0, 22021.0, -22.965490500115564, -43.174725170148236, 13473.0, -22.72841826089216, -47.301218172629724))"
17.54330525544772,18.65,"List(1, 14, List(), List(990.0, 1.528673858E9, 1.5288129E9, 110.32, 128.97, 200.0, 42.0, 35.0, 88301.0, -26.90558514542844, -48.66208319734036, 13186.0, -22.881923999999998, -47.210386499999984))"
9.586066563042879,12.85,"List(1, 14, List(), List(41440.0, 1.503268498E9, 1.503347738E9, 128.5, 141.35, 1550.0, 35.0, 35.0, 5303.0, -23.52666051012755, -46.7315398082117, 12940.0, -23.115630732734584, -46.55374249411723))"
15.412341628959304,15.1,"List(1, 14, List(), List(2112.0, 1.51214402E9, 1.512435781E9, 45.65, 60.75, 200.0, 43.0, 35.0, 96760.0, -30.67428812160817, -51.39379644684868, 14092.0, -21.18135975131287, -47.76213561281669))"
17.165794663573088,25.3,"List(1, 14, List(), List(23625.0, 1.516135104E9, 1.516226917E9, 65.9, 91.2, 800.0, 43.0, 35.0, 95900.0, -29.460893622249376, -51.964232704046054, 14940.0, -21.76590242683773, -48.824481207427674))"


In [62]:
evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("MAE = %g" % mae)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R2 = %g" % r2)

maeAR_teste  = mae
rmseAR_teste = rmse
r2AR_teste   = r2

treeModel = modelarvore.stages[1]
#Base cheia
#----------------------------------------------------------------------------
evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_Tot)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions_Tot)
print("MAE = %g" % mae)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions_Tot)
print("R2 = %g" % r2)

maeAR_teste_Tot  = mae
rmseAR_teste_Tot = rmse
r2AR_teste_Tot   = r2

Root Mean Squared Error (RMSE) on test data = 3.65413
MAE = 2.4755
R2 = 0.569128
Root Mean Squared Error (RMSE) on test data = 10.2471
MAE = 5.29215
R2 = 0.582525

In [63]:
display(treeModel)

treeNode
"{""index"":31,""featureType"":""categorical"",""prediction"":null,""threshold"":null,""categories"":[19.0],""feature"":6,""overflow"":false}"
"{""index"":15,""featureType"":""continuous"",""prediction"":null,""threshold"":19801.5,""categories"":null,""feature"":11,""overflow"":false}"
"{""index"":7,""featureType"":""continuous"",""prediction"":null,""threshold"":1848.0,""categories"":null,""feature"":5,""overflow"":false}"
"{""index"":3,""featureType"":""continuous"",""prediction"":null,""threshold"":13504.5,""categories"":null,""feature"":11,""overflow"":false}"
"{""index"":1,""featureType"":""continuous"",""prediction"":null,""threshold"":-47.67287965602982,""categories"":null,""feature"":10,""overflow"":false}"
"{""index"":0,""featureType"":null,""prediction"":12.952398109864147,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":2,""featureType"":null,""prediction"":9.586066563042879,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":5,""featureType"":""continuous"",""prediction"":null,""threshold"":48.995000000000005,""categories"":null,""feature"":3,""overflow"":false}"
"{""index"":4,""featureType"":null,""prediction"":12.01333749220212,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":6,""featureType"":null,""prediction"":13.364283555018144,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"


In [65]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=100).fit(df_assembler)

featureIndexer_Tot =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=100).fit(df_assembler_Tot)

# Train a GBT model.
gbt = GBTRegressor(featuresCol="indexedFeatures", labelCol="freight_value",  maxIter=10)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])
pipeline_Tot = Pipeline(stages=[featureIndexer_Tot, gbt])

# Train model.  This also runs the indexer.
modelgbt = pipeline.fit(treino)
modelgbt_Tot = pipeline_Tot.fit(treino_Tot)

predictions = modelgbt.transform(teste)
predictions_Tot = modelgbt_Tot.transform(teste_Tot)

In [66]:
display(predictions.select("prediction", "freight_value", "features"))

prediction,freight_value,features
9.362100123096646,11.73,"List(1, 14, List(), List(20000.0, 1.518045025E9, 1.518207135E9, 65.9, 77.63, 250.0, 35.0, 35.0, 1151.0, -23.53250199140166, -46.655183398840066, 5767.0, -23.633090317589396, -46.759997268731205))"
17.923280101636944,21.47,"List(1, 14, List(), List(14350.0, 1.493826149E9, 1.493828237E9, 119.99, 141.46, 1350.0, 42.0, 31.0, 89287.0, -26.237652633823842, -49.38599561257329, 37564.0, -22.27164789897835, -46.16347079455195))"
16.969540728565477,15.56,"List(1, 14, List(), List(7140.0, 1.493831729E9, 1.494243743E9, 39.0, 54.56, 1000.0, 41.0, 35.0, 80050.0, -25.43482090674591, -49.25340082846679, 12327.0, -23.30593717675828, -45.97121752309212))"
14.611575529728167,15.1,"List(1, 14, List(), List(576.0, 1.499445816E9, 1.499519264E9, 35.0, 50.1, 150.0, 35.0, 31.0, 13566.0, -21.999410549541242, -47.897242066478114, 38440.0, -18.650389455353206, -48.19675571936371))"
12.817349030586648,11.85,"List(1, 14, List(), List(23625.0, 1.498756808E9, 1.498837575E9, 48.9, 60.75, 900.0, 35.0, 35.0, 13504.0, -22.392499159039716, -47.573597665156555, 14940.0, -21.76590242683773, -48.824481207427674))"
17.541348593360443,16.76,"List(1, 14, List(), List(15360.0, 1.519378176E9, 1.519420964E9, 72.9, 89.66, 1500.0, 33.0, 35.0, 22021.0, -22.965490500115564, -43.174725170148236, 13473.0, -22.72841826089216, -47.301218172629724))"
17.642586739398617,18.65,"List(1, 14, List(), List(990.0, 1.528673858E9, 1.5288129E9, 110.32, 128.97, 200.0, 42.0, 35.0, 88301.0, -26.90558514542844, -48.66208319734036, 13186.0, -22.881923999999998, -47.210386499999984))"
10.443179019995588,12.85,"List(1, 14, List(), List(41440.0, 1.503268498E9, 1.503347738E9, 128.5, 141.35, 1550.0, 35.0, 35.0, 5303.0, -23.52666051012755, -46.7315398082117, 12940.0, -23.115630732734584, -46.55374249411723))"
15.005014848476488,15.1,"List(1, 14, List(), List(2112.0, 1.51214402E9, 1.512435781E9, 45.65, 60.75, 200.0, 43.0, 35.0, 96760.0, -30.67428812160817, -51.39379644684868, 14092.0, -21.18135975131287, -47.76213561281669))"
17.677578608750995,25.3,"List(1, 14, List(), List(23625.0, 1.516135104E9, 1.516226917E9, 65.9, 91.2, 800.0, 43.0, 35.0, 95900.0, -29.460893622249376, -51.964232704046054, 14940.0, -21.76590242683773, -48.824481207427674))"


In [67]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("MAE = %g" % mae)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R2 = %g" % r2)

maeGBT_teste  = mae
rmseGBT_teste = rmse
r2GBT_teste   = r2

gbtModel = modelgbt.stages[1]
print(gbtModel)  # summary only

#Base Cheia
#------------------------------------------------------------------------------
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions_Tot)
print("MAE = %g" % mae)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_Tot)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions_Tot)
print("R2 = %g" % r2)

maeGBT_teste_Tot  = mae
rmseGBT_teste_Tot = rmse
r2GBT_teste_Tot   = r2


MAE = 2.20676
Root Mean Squared Error (RMSE) on test data = 3.35346
R2 = 0.637117
GBTRegressionModel (uid=GBTRegressor_3dde56f76637) with 10 trees
MAE = 4.58334
Root Mean Squared Error (RMSE) on test data = 9.17618
R2 = 0.665223

In [69]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=100).fit(df_assembler)

featureIndexer_Tot =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=100).fit(df_assembler_Tot)

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures", labelCol="freight_value")

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])
pipeline_Tot = Pipeline(stages=[featureIndexer_Tot, rf])

# Train model.  This also runs the indexer.
modelrandom = pipeline.fit(treino)
modelrandom_Tot = pipeline_Tot.fit(treino_Tot)

# Make predictions.
predictions = modelrandom.transform(teste)
predictions_Tot = modelrandom_Tot.transform(teste_Tot)



In [70]:
display(predictions.select("prediction", "freight_value", "features"))

prediction,freight_value,features
10.555084358418734,11.73,"List(1, 14, List(), List(20000.0, 1.518045025E9, 1.518207135E9, 65.9, 77.63, 250.0, 35.0, 35.0, 1151.0, -23.53250199140166, -46.655183398840066, 5767.0, -23.633090317589396, -46.759997268731205))"
17.580823675815232,21.47,"List(1, 14, List(), List(14350.0, 1.493826149E9, 1.493828237E9, 119.99, 141.46, 1350.0, 42.0, 31.0, 89287.0, -26.237652633823842, -49.38599561257329, 37564.0, -22.27164789897835, -46.16347079455195))"
16.725360621078927,15.56,"List(1, 14, List(), List(7140.0, 1.493831729E9, 1.494243743E9, 39.0, 54.56, 1000.0, 41.0, 35.0, 80050.0, -25.43482090674591, -49.25340082846679, 12327.0, -23.30593717675828, -45.97121752309212))"
15.659930694478708,15.1,"List(1, 14, List(), List(576.0, 1.499445816E9, 1.499519264E9, 35.0, 50.1, 150.0, 35.0, 31.0, 13566.0, -21.999410549541242, -47.897242066478114, 38440.0, -18.650389455353206, -48.19675571936371))"
12.971302640099609,11.85,"List(1, 14, List(), List(23625.0, 1.498756808E9, 1.498837575E9, 48.9, 60.75, 900.0, 35.0, 35.0, 13504.0, -22.392499159039716, -47.573597665156555, 14940.0, -21.76590242683773, -48.824481207427674))"
17.245252862847842,16.76,"List(1, 14, List(), List(15360.0, 1.519378176E9, 1.519420964E9, 72.9, 89.66, 1500.0, 33.0, 35.0, 22021.0, -22.965490500115564, -43.174725170148236, 13473.0, -22.72841826089216, -47.301218172629724))"
18.410145569582863,18.65,"List(1, 14, List(), List(990.0, 1.528673858E9, 1.5288129E9, 110.32, 128.97, 200.0, 42.0, 35.0, 88301.0, -26.90558514542844, -48.66208319734036, 13186.0, -22.881923999999998, -47.210386499999984))"
13.583911388051993,12.85,"List(1, 14, List(), List(41440.0, 1.503268498E9, 1.503347738E9, 128.5, 141.35, 1550.0, 35.0, 35.0, 5303.0, -23.52666051012755, -46.7315398082117, 12940.0, -23.115630732734584, -46.55374249411723))"
15.849764361473442,15.1,"List(1, 14, List(), List(2112.0, 1.51214402E9, 1.512435781E9, 45.65, 60.75, 200.0, 43.0, 35.0, 96760.0, -30.67428812160817, -51.39379644684868, 14092.0, -21.18135975131287, -47.76213561281669))"
16.886781801677575,25.3,"List(1, 14, List(), List(23625.0, 1.516135104E9, 1.516226917E9, 65.9, 91.2, 800.0, 43.0, 35.0, 95900.0, -29.460893622249376, -51.964232704046054, 14940.0, -21.76590242683773, -48.824481207427674))"


In [71]:
evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("MAE = %g" % mae)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R2 = %g" % r2)

maeRFR_teste  = mae
rmseRFR_teste = rmse
r2RFR_teste   = r2

rfModel = modelrandom.stages[1]
print(rfModel)  # summary only

#Base cheia
#----------------------------------------------------------------------------

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_Tot)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions_Tot)
print("MAE = %g" % mae)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions_Tot)
print("R2 = %g" % r2)

maeRFR_teste_Tot  = mae
rmseRFR_teste_Tot = rmse
r2RFR_teste_Tot   = r2


Root Mean Squared Error (RMSE) on test data = 3.54925
MAE = 2.43869
R2 = 0.593508
RandomForestRegressionModel (uid=RandomForestRegressor_fb2cee489dba) with 20 trees
Root Mean Squared Error (RMSE) on test data = 9.82119
MAE = 4.97975
R2 = 0.616506

In [73]:
from pyspark.ml.regression import GeneralizedLinearRegression

glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3, labelCol="freight_value")

# Fit the model
modelgau = glr.fit(treino)
modelgau_Tot = glr.fit(treino_Tot)

# Print the coefficients and intercept for generalized linear regression model
print("Coefficients: " + str(modelgau.coefficients))
print("Intercept: " + str(modelgau.intercept))

# Summarize the model over the training set and print out some metrics
summary = modelgau.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()

Coefficients: [4.9414914566149814e-05,1.4790511382879692e-08,1.487664663016236e-08,0.007046465821939837,0.0009802010331238487,0.0005137803622216509,-0.11430836052470711,-0.06467934377392949,7.947194107704459e-05,0.22048291039743997,0.15356728372074702,4.364746123020856e-05,0.16545151987560544,-0.12746844743807315]
Intercept: -18.619316181133385
Coefficient Standard Errors: [1.288135712576668e-06, 3.548570594658854e-09, 3.5550313237880955e-09, 0.00018042500788695434, 0.00014231983047577592, 8.249759024602532e-06, 0.0033750003887049007, 0.006455542632188849, 5.933697271635635e-07, 0.0037095179308253416, 0.005095495405720412, 6.932632899013789e-07, 0.006628659445256777, 0.008732872577458202, 1.7818800415015097]
T Values: [38.36157485868067, 4.1680194851249945, 4.184673853818654, 39.054817868456546, 6.887311696810147, 62.27822663540218, -33.8691399584019, -10.019195512926073, 133.93325853837837, 59.43707902454702, 30.137851473351557, 62.959429506815056, 24.960027173216208, -14.596393833468051, -10.449253455605087]
P Values: [0.0, 3.076567785598172e-05, 2.859524260623303e-05, 0.0, 5.737410546657884e-12, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Dispersion: 15.253013898550215
Null Deviance: 1983677.9052049462
Residual Degree Of Freedom Null: 64688
Deviance: 986473.4208748366
Residual Degree Of Freedom: 64674
AIC: 359859.5346806518
Deviance Residuals: 
+--------------------+
 devianceResiduals|
+--------------------+
 1.626092033467696|
 -3.6823604783755783|
 10.133513494492075|
 0.440790610176677|
 4.703041683479821|
 -2.6475293496657315|
 -5.561137039954318|
 -0.964271295584533|
 -1.31381917203578|
 -3.1428553816204507|
-0.02713177763989...|
 0.11079971546199019|
 -2.7608534840155645|
 0.2867525678488754|
 2.124672990554643|
-0.02806769045586...|
 1.8407161024201315|
 -2.955296980780987|
 -0.9783876740557123|
 0.14702886773599566|
+--------------------+
only showing top 20 rows

In [74]:
predictions = modelgau.transform(teste)
predictions_Tot = modelgau_Tot.transform(teste_Tot)

In [75]:
display(predictions.select("prediction", "freight_value", "features"))

prediction,freight_value,features
11.852062275627006,11.73,"List(1, 14, List(), List(20000.0, 1.518045025E9, 1.518207135E9, 65.9, 77.63, 250.0, 35.0, 35.0, 1151.0, -23.53250199140166, -46.655183398840066, 5767.0, -23.633090317589396, -46.759997268731205))"
18.845017851157664,21.47,"List(1, 14, List(), List(14350.0, 1.493826149E9, 1.493828237E9, 119.99, 141.46, 1350.0, 42.0, 31.0, 89287.0, -26.237652633823842, -49.38599561257329, 37564.0, -22.27164789897835, -46.16347079455195))"
15.681023944831281,15.56,"List(1, 14, List(), List(7140.0, 1.493831729E9, 1.494243743E9, 39.0, 54.56, 1000.0, 41.0, 35.0, 80050.0, -25.43482090674591, -49.25340082846679, 12327.0, -23.30593717675828, -45.97121752309212))"
13.869296256293758,15.1,"List(1, 14, List(), List(576.0, 1.499445816E9, 1.499519264E9, 35.0, 50.1, 150.0, 35.0, 31.0, 13566.0, -21.999410549541242, -47.897242066478114, 38440.0, -18.650389455353206, -48.19675571936371))"
13.719869616331557,11.85,"List(1, 14, List(), List(23625.0, 1.498756808E9, 1.498837575E9, 48.9, 60.75, 900.0, 35.0, 35.0, 13504.0, -22.392499159039716, -47.573597665156555, 14940.0, -21.76590242683773, -48.824481207427674))"
15.46560757693134,16.76,"List(1, 14, List(), List(15360.0, 1.519378176E9, 1.519420964E9, 72.9, 89.66, 1500.0, 33.0, 35.0, 22021.0, -22.965490500115564, -43.174725170148236, 13473.0, -22.72841826089216, -47.301218172629724))"
17.14473969755312,18.65,"List(1, 14, List(), List(990.0, 1.528673858E9, 1.5288129E9, 110.32, 128.97, 200.0, 42.0, 35.0, 88301.0, -26.90558514542844, -48.66208319734036, 13186.0, -22.881923999999998, -47.210386499999984))"
14.335325634056602,12.85,"List(1, 14, List(), List(41440.0, 1.503268498E9, 1.503347738E9, 128.5, 141.35, 1550.0, 35.0, 35.0, 5303.0, -23.52666051012755, -46.7315398082117, 12940.0, -23.115630732734584, -46.55374249411723))"
15.888242150287994,15.1,"List(1, 14, List(), List(2112.0, 1.51214402E9, 1.512435781E9, 45.65, 60.75, 200.0, 43.0, 35.0, 96760.0, -30.67428812160817, -51.39379644684868, 14092.0, -21.18135975131287, -47.76213561281669))"
17.734842694378578,25.3,"List(1, 14, List(), List(23625.0, 1.516135104E9, 1.516226917E9, 65.9, 91.2, 800.0, 43.0, 35.0, 95900.0, -29.460893622249376, -51.964232704046054, 14940.0, -21.76590242683773, -48.824481207427674))"


In [76]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("MAE = %g" % mae)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R2 = %g" % r2)

maeGLR_Gau_teste  = mae
rmseGLR_Gau_teste = rmse
r2GLR_Gau_teste   = r2

#Base cheia
#----------------------------------------------------------------------------

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_Tot)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions_Tot)
print("MAE = %g" % mae)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions_Tot)
print("R2 = %g" % r2)

maeGLR_Gau_teste_Tot  = mae
rmseGLR_Gau_teste_Tot = rmse
r2GLR_Gau_teste_Tot   = r2



Root Mean Squared Error (RMSE) on test data = 3.97204
MAE = 2.85142
R2 = 0.490896
Root Mean Squared Error (RMSE) on test data = 10.0046
MAE = 5.45579
R2 = 0.602052

In [78]:
columns = ("MODELO", "MAE", "MAE_FULL", "RMSE", "RMSE_FULL", "R2", "R2_FULL")
data    = ( ('LinearRegression'                   , maeLR_teste     , maeLR_teste_Tot     , rmseLR_teste      , rmseLR_teste_Tot      , r2LR_teste       * 100,   r2LR_teste_Tot       * 100),
           ('DecisionTreeRegressor'               , maeAR_teste     , maeAR_teste_Tot     , rmseAR_teste      , rmseAR_teste_Tot      , r2AR_teste       * 100,   r2AR_teste_Tot       * 100),
           ('GBTRegressor'                        , maeGBT_teste    , maeGBT_teste_Tot    , rmseGBT_teste     , rmseGBT_teste_Tot     , r2GBT_teste      * 100,   r2GBT_teste_Tot      * 100),
           ('RandomForestRegressor'               , maeRFR_teste    , maeRFR_teste_Tot    , rmseRFR_teste     , rmseRFR_teste_Tot     , r2RFR_teste      * 100,   r2RFR_teste_Tot      * 100),
           ('GeneralizedLinearRegression Gaussian', maeGLR_Gau_teste, maeGLR_Gau_teste_Tot, rmseGLR_Gau_teste , rmseGLR_Gau_teste_Tot , r2GLR_Gau_teste  * 100,   r2GLR_Gau_teste_Tot  * 100))

rdd = spark.sparkContext.parallelize(data)
dfFromRDD1 = rdd.toDF()
dfFromRDD1 = rdd.toDF(columns)

display(dfFromRDD1.orderBy('R2', ascending=False, ))

MODELO,MAE,MAE_FULL,RMSE,RMSE_FULL,R2,R2_FULL
GBTRegressor,2.2067552295501085,4.583344871479707,3.353462665399888,9.176182656644267,63.711742822974585,66.52232450265589
RandomForestRegressor,2.4386871200881983,4.979750210546967,3.5492491211318056,9.821185727508267,59.35078843421605,61.65055847756098
DecisionTreeRegressor,2.475496142165073,5.292154050781435,3.6541334069891223,10.247068969984777,56.91283131217906,58.252496060545546
LinearRegression,2.8392927715351233,5.479060500079389,3.9651163473027546,10.002473196291746,49.24131588730973,60.2037358457719
GeneralizedLinearRegression Gaussian,2.851415100850553,5.455792358832191,3.9720395242939763,10.00455563950751,49.089626621715766,60.20515628672596


In [79]:
'''
MODELO	                                        MAE	             MAE_FULL	           RMSE	             RMSE_FULL	          R2	             R2_FULL
GBTRegressor	                        2.0971217666366444	4.583344871479707	3.1085756587732503	9.176182656644267	67.35261187217321	66.52232450265589
RandomForestRegressor	                2.3674106055636184	4.979750210546967	3.351403732253019	9.821185727508267	62.05285941173548	61.65055847756098
DecisionTreeRegressor	                2.372980855745172	5.292154050781435	3.4319405710015305	10.247068969984777	60.20714714705073	58.252496060545546
LinearRegression	                    2.7621046346692912	5.479060500079389	3.8068325473911027	10.002473196291746	51.013786374337265	60.2037358457719
GeneralizedLinearRegression Gaussian	2.774380434051552	5.455792358832191	3.8122327100255298	10.00455563950751	50.899675182932015	60.20515628672596

'''

In [80]:
#Sendo assim, incluo a feature product_category_name_Index para comparação dos resultados deste modelo em especifico, por se tratar do melhor modelo utilizado nesta analise
df = data_merge.select('product_category_name',
               'volume',               
               'order_approved_at',
               'order_delivered_carrier_date',
               'price', 
               'payment_value',                
               'product_weight_g', 
               'codigo_UF_customers',
               'codigo_UF_sellers',
               'customer_zip_code_prefix',
               'geolocation_lat_customer',
               'geolocation_lng_customer',
               'seller_zip_code_prefix',
               'geolocation_lat_seller',
               'geolocation_lng_seller',
               'freight_value'
              )  

df = df.repartition(2).cache()
df.count()

from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
#para utilização do campo no modelo de regressao, faço uma tratativa nesta feature pois a mesma esta em formato string, sendo que o modelo exige variaveis numericas
indexer = StringIndexer().setInputCol("product_category_name").setOutputCol("product_category_name_Index").fit(df)
df = indexer.transform(df)

df = df.select('product_category_name_Index',
               'volume',               
               'order_approved_at',
               'order_delivered_carrier_date',
               'price', 
               'payment_value',                
               'product_weight_g', 
               'codigo_UF_customers',
               'codigo_UF_sellers',
               'customer_zip_code_prefix',
               'geolocation_lat_customer',
               'geolocation_lng_customer',
               'seller_zip_code_prefix',
               'geolocation_lat_seller',
               'geolocation_lng_seller',
               'freight_value'
              )  


df = df.repartition(2).cache()
df.count()


Out[363]: 114464

In [81]:
data_merge_total = df

data_merge_total = data_merge_total.repartition(2).cache()
data_merge_total.count()

data_merge_Slice = data_merge_total.where((data_merge_total['freight_value'] > nMin) & (data_merge_total['freight_value'] < nMax))

data_merge_Slice = data_merge_Slice.repartition(2).cache()
data_merge_Slice.count()

#Antes verifico a quantidade registros que serão descartados
print(data_merge_total.count() - data_merge_Slice.count())

df = data_merge_Slice

11929

Entendendo que mesmo utilizando apenas os dados até o 3-Quartil, ainda possuo o mesmo numero de categorias de produto na base

In [83]:
df1 = (data_merge_total.select('*').groupBy('product_category_name_Index').count())
df1 = (df1.orderBy('count', ascending=False))
display(df1.select('*').limit(20))


product_category_name_Index,count
0.0,11622
1.0,9733
2.0,8715
3.0,8534
4.0,7876
5.0,7154
6.0,6039
7.0,4570
8.0,4452
9.0,4273


In [84]:
dfC1  = (df1.select('*').groupBy('product_category_name_Index').count())
print(dfC1.select('count').count())

74

In [85]:
df2 =(data_merge_Slice.select('*').where((data_merge_Slice['freight_value'] > 1) & (data_merge_Slice['freight_value'] < 34.13)).groupBy('product_category_name_Index').count())
df2 = (df2.orderBy('count', ascending=False))
display(df2.select('*').limit(20))

product_category_name_Index,count
0.0,10974
1.0,9010
2.0,8042
3.0,7468
4.0,7100
5.0,6359
6.0,5496
7.0,4380
10.0,3833
9.0,3738


In [86]:
#Verificamos que fazem parte desta base, os mesmas categorias de produto da base total.
dfC2  = (df2.select('*').groupBy('product_category_name_Index').count())
print(dfC2.select('count').count())

74

In [87]:
assemblerGBT = VectorAssembler(inputCols = df.columns[:-1], outputCol = 'features')
assemblerGBT_tot = VectorAssembler(inputCols = data_merge_total.columns[:-1], outputCol = 'features')

assemblerGBT = assemblerGBT.transform(df)
assemblerGBT_Tot = assemblerGBT_tot.transform(data_merge_total)
#display(assemblerGBT)

In [88]:
treinoGBT, testeGBT = assemblerGBT .randomSplit([0.7, 0.3], seed=36)
treinoGBT_Tot, testeGBT_Tot = assemblerGBT_Tot .randomSplit([0.7, 0.3], seed=36)

treinoGBT = treinoGBT.repartition(2).cache()
treinoGBT.count()

testeGBT = testeGBT.repartition(2).cache()
testeGBT.count()

treinoGBT_Tot = treinoGBT_Tot.repartition(2).cache()
treinoGBT_Tot.count()

testeGBT_Tot = testeGBT_Tot.repartition(2).cache()
testeGBT_Tot.count()

Out[370]: 34236

In [89]:
#GRADIENTE BOOST TREE REGRESSION

featureIndexerGBT =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=100).fit(assemblerGBT)

featureIndexerGBT_Tot =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=100).fit(assemblerGBT_Tot)

# Train a GBT model.
#Setada a variavel maxBins para 74, pois possuimos 74 categorias de produto e o GBTR aceita por padrao 32
gbtPCN = GBTRegressor(featuresCol="indexedFeatures", labelCol="freight_value",  maxIter=10, maxBins=74)

# Chain indexer and GBT in a Pipeline
pipelineGBT = Pipeline(stages=[featureIndexerGBT, gbtPCN])
pipelineGBT_Tot = Pipeline(stages=[featureIndexerGBT_Tot, gbtPCN])

# Train model.  This also runs the indexer.
modelgbtPCA = pipelineGBT.fit(treinoGBT)
modelgbtPCA_Tot = pipelineGBT_Tot.fit(treinoGBT_Tot)

predictionsGBT = modelgbtPCA.transform(testeGBT)
predictionsGBT_Tot = modelgbtPCA.transform(testeGBT_Tot)

predictionsGBT.select("prediction", "freight_value", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictionsGBT)
print("MAE = %g" % mae)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictionsGBT)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictionsGBT)
print("R2 = %g" % r2)

maeGBT_PCA_teste  = mae
rmseGBT_PCA_teste = rmse
r2GBT_PCA_teste   = r2

gbtModel = modelgbtPCA.stages[1]
print(gbtModel)  # summary only

#Base cheia
#----------------------------------------------------------------------------

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictionsGBT_Tot)
print("MAE = %g" % mae)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictionsGBT_Tot)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="freight_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictionsGBT_Tot)
print("R2 = %g" % r2)

maeGBT_PCA_teste_Tot  = mae
rmseGBT_PCA_teste_Tot = rmse
r2GBT_PCA_teste_Tot   = r2

+------------------+-------------+--------------------+
 prediction|freight_value| features|
+------------------+-------------+--------------------+
15.649837559535198| 18.34|[2.0,27000.0,1.53...|
15.357877630639596| 16.53|[2.0,7260.0,1.516...|
13.829966747150515| 11.9|[0.0,18000.0,1.50...|
 27.79595816530316| 28.82|[8.0,19800.0,1.50...|
14.966478949909607| 14.11|[10.0,8000.0,1.51...|
+------------------+-------------+--------------------+
only showing top 5 rows

MAE = 2.17966
Root Mean Squared Error (RMSE) on test data = 3.28467
R2 = 0.651697
GBTRegressionModel (uid=GBTRegressor_16e6f405026e) with 10 trees
MAE = 5.04762
Root Mean Squared Error (RMSE) on test data = 13.6062
R2 = 0.246315
Out[371]: 'MAE = 2.12244\nRoot Mean Squared Error (RMSE) on test data = 3.16462\nR2 = 0.661097\nGBTRegressionModel (uid=GBTRegressor_7c35253833a6) with 10 trees\nMAE = 5.02512\nRoot Mean Squared Error (RMSE) on test data = 13.5152\nR2 = 0.256363'

In [90]:
columns = ("MODELO", "MAE", "MAE_FULL", "RMSE", "RMSE_FULL", "R2", "R2_FULL")
data    = (('GBTRegressor'                        , maeGBT_teste    , maeGBT_teste_Tot    , rmseGBT_teste     , rmseGBT_teste_Tot     , r2GBT_teste      * 100,   r2GBT_teste_Tot      * 100),
           ('GBTRegressor Product_Category_Name'  , maeGBT_PCA_teste, maeGBT_PCA_teste_Tot, rmseGBT_PCA_teste , rmseGBT_PCA_teste_Tot , r2GBT_PCA_teste  * 100,  r2GBT_PCA_teste_Tot  * 100))


rdd2 = spark.sparkContext.parallelize(data)
dfFromRDD2 = rdd2.toDF()
dfFromRDD2 = rdd2.toDF(columns)

display(dfFromRDD2.orderBy('R2', ascending=False ))

MODELO,MAE,MAE_FULL,RMSE,RMSE_FULL,R2,R2_FULL
GBTRegressor Product_Category_Name,2.179660325222447,5.047618348737022,3.2846722285807144,13.6061655694988,65.16973563795916,24.631456381645112
GBTRegressor,2.2067552295501085,4.583344871479707,3.353462665399888,9.176182656644267,63.711742822974585,66.52232450265589


In [91]:
'''
MODELO	                                        MAE	             MAE_FULL	           RMSE	             RMSE_FULL	          R2	             R2_FULL
GBTRegressor	                    2.0971217666366444	4.583344871479707	3.1085756587732503	9.176182656644267	67.35261187217321	66.52232450265589
GBTRegressor Product_Category_Name	2.1224359773210923	5.025119259065538	3.1646246530457445	13.515160270905735	66.10970512026672	25.636294764091193
'''


FIM DO NOTEBOOK